In [1]:
import numpy as np
import glob
import os
import zipfile as zf
import pickle
from pathlib import Path
import random
import math
import scipy.ndimage
import scipy.io
import matplotlib.pyplot as plt
import matplotlib.image as img
import sys

# global variables:
MIN_MATCH_COUNT = 24 # minimum number of matches required
MAX_ITER = 5000 # number of iterations in ransac
NUM_MATCH = 500 # total number of matches that we want to subsample from different locations of the image.
                # For every region in the image analysed, if it contains at least a descriptor, and 
                # if the number of descriptors subsampled is fixed to N = 1, there will be 500 matches. 
                # Nevertheless, some regions of the image can have no descriptors, so the total number of matches will be reduced. 
                # For solving this, we can sample N > 1 descriptors from the regions of the image analysed 

subsampling = True # variable that tunes the subsampling or not of the souce image descriptors, 
                   # when performing the matching 
    
threshold_error = 4 # threshold error for points to be considered inliers, when performing ransac 

In [2]:
def match_features(des_src, des_dest, threshold):
    '''
    Implements the Nearest Neighbor Distance Ratio Test (NNDR) - Equation 4.18 in Section 4.1.3 of 
    Szeliski - to assign matches between interest points in two images. It also searches for mutual 
    matches and applies the NNDR test
  
    A match is between a feature in im1_features and a feature in im2_features. We can
    represent this match as a the index of the feature in im1_features and the index
    of the feature in im2_features
    
    :params:
    :des_src: an np array of features for interest points in source image
    :des_dest: an np array of features for interest points in destination image
    
    :returns:
    :matches: an np array of dimension k x 2 where k is the number of matches. The first
            column is an index into im1_features and the second column is an index into im2_features
    '''
    
    global MIN_MATCH_COUNT

    matches = []
    
    # Re-normalize
    des_dest_normalize = des_dest / np.linalg.norm(des_dest, axis = 0)
    
    des_src_normalize = des_src / np.linalg.norm(des_src, axis = 0)
    
    # cosine similarity (descriptors are L2 normalized) 
    matrix_similarity = des_src_normalize.T @ des_dest_normalize
    
    ind_col_matches = np.argmax(matrix_similarity, axis = 1)
        
    matches = np.concatenate((np.arange(0, des_src.shape[1]).reshape(-1,1), ind_col_matches.reshape(-1, 1)), axis = 1)
    final_matches = matches
    
    # FIND GOOD MATCHES:
    # Retrieve top 2 nearest neighbors 1->2.
    index_sorted = np.argsort(-matrix_similarity, axis = 1)[:, 0:2]

    matrix_distances = np.sqrt(2 - 2 * matrix_similarity)
    
    mask_good_matches = matrix_distances[list(range(0,matrix_distances.shape[0])), index_sorted[:, 0]] / matrix_distances[list(range(0,matrix_distances.shape[0])), index_sorted[:, 1]] < threshold
    
    if np.any(mask_good_matches):
        good_matches = matches[mask_good_matches, :]
        
        print("good matches/matches - %d/%d" % (good_matches.shape[0],matches.shape[0]))
        
        if good_matches.shape[0] > MIN_MATCH_COUNT:
            final_matches = good_matches
    
    # FIND MUTUAL AND GOOD MATCHES: 
    # Retrieve top 2 nearest neighbors 1->2.
    matches_12_top2 = np.argsort(-matrix_similarity, axis = 1)[:, 0:2]
    matches_12 = matches_12_top2[:, 0] # Save first NN and match similarity.
    
    matrix_distances = np.sqrt(2 - 2 * matrix_similarity)
    
    # Compute Lowe's ratio.
    mask1_good_matches = matrix_distances[list(range(0,matrix_distances.shape[0])), matches_12_top2[:, 0]] / matrix_distances[list(range(0,matrix_distances.shape[0])), matches_12_top2[:, 1]] < threshold

    # Retrieve top 2 nearest neighbors 1->2.
    matches_21_top2 = np.argsort(-matrix_similarity.T, axis = 1)[:, 0:2]
    matches_21 = matches_21_top2[:, 0] # Save first NN and match similarity.
    
    matrix_distances_T = np.sqrt(2 - 2 * matrix_similarity.T)
    
    # Compute Lowe's ratio.
    mask2_good_matches = matrix_distances_T[list(range(0,matrix_distances_T.shape[0])), matches_21_top2[:, 0]] / matrix_distances_T[list(range(0,matrix_distances_T.shape[0])), matches_21_top2[:, 1]] < threshold
        
    final_mask_good_matches = mask1_good_matches & mask2_good_matches[matches_12]
    
    # Mutual NN + symmetric ratio test.
    ids1 = np.arange(0, matrix_similarity.shape[0])
    
    mask_mutual_matches = (ids1 == matches_21[matches_12]) & final_mask_good_matches
    
    if np.any(mask_mutual_matches):
        mutual_matches = matches[mask_mutual_matches, :]
        
        if mutual_matches.shape[0] > MIN_MATCH_COUNT:
            final_matches = mutual_matches
        
        print("mutual and good matches/matches - %d/%d" % (mutual_matches.shape[0],matches.shape[0]))

    return final_matches

In [3]:
def siftMatch(img1, sift_path_ref, sift_path_image, threshold = 0.75, N = 1):
    
    global NUM_MATCH, subsampling

    data_ref = scipy.io.loadmat(sift_path_ref)
    dst = data_ref['p'] # (2,N) numpy array, where N is the total number of keypoints
    des_dest = data_ref['d'] # (128,N) numpy array, where N is the total number of keypoints
        
    data_image = scipy.io.loadmat(sift_path_image)
    src = data_image['p'] # (2,N) numpy array, where N is the total number of keypoints
    des_src = data_image['d'] # (128,N) numpy array, where N is the total number of keypoints
        
    if subsampling:
        h, w, _ = img1.shape
            
        h_subsampling = math.floor(h/4) 
            
        w_subsampling = math.floor(w * h/(NUM_MATCH*h_subsampling)) 
            
        regions_h = range(0, h+1, h_subsampling)
        regions_w = range(0, w+1, w_subsampling)
            
        des_src_subsampling = np.array([], dtype=np.int64).reshape(des_src.shape[0],0)
        src_subsampling = np.array([], dtype=np.int64).reshape(src.shape[0],0)
            
        id_descriptor = np.arange(des_src.shape[1])
            
        for i in range(len(regions_h)-1):
            h_region_min = regions_h[i]
            h_region_max = regions_h[i+1]-1
            
            for j in range(len(regions_w)-1):
                w_region_min = regions_w[j]
                w_region_max = regions_w[j+1]-1
                    
                ind_keypoints_region = (src[0,:] > w_region_min) & (src[0,:] < w_region_max) & (src[1,:] > h_region_min) & (src[1,:] < h_region_max)
                    
                if np.any(ind_keypoints_region):
                    if len(ind_keypoints_region[ind_keypoints_region == True]) < N:
                        num_sampling = len(ind_keypoints_region[ind_keypoints_region == True])
                            
                    else:
                        num_sampling = N 
                        
                    ind_d_des = random.sample(list(id_descriptor[ind_keypoints_region]), num_sampling)
                        
                    des_src_subsampling = np.concatenate((des_src_subsampling, des_src[:, ind_d_des]), axis = 1)
                    src_subsampling = np.concatenate((src_subsampling, src[:, ind_d_des]), axis = 1)
            
        des_src = des_src_subsampling
        src = src_subsampling
            
    m =match_features(des_src,des_dest, threshold)
        
    matches_coords = np.concatenate((src[:, m[:,0]], dst[:, m[:, 1]]))
        
    src_pts = matches_coords[0:2, :].T
    dst_pts = matches_coords[2:4, :].T
            
    return src_pts, dst_pts

In [4]:
def FitHomography(selected_matches, N = 4):
    """ Compute the fitted homography matrix by using N match pairs
  
   [u]     [X]
   [v] = H [Y], 
   [1]     [1]
   being H a 3x3 matrix 
  
   This can be arranged in a system Ax = 0, where x is a column vector with 
   the parameters of the homography, and A is given by:
   A = [X Y 1 0 0 0 -u.X -u.Y -u]
       [0 0 0 X Y 1 -v.X -v.Y -v]

   For N matches, the above matrix is vertically stacked, with 2 rows per match 
  """
    
    X = selected_matches[:,0]
    Y = selected_matches[:,1]
    u = selected_matches[:,2]
    v = selected_matches[:,3]
    
    A = []
    
    for i in range(N):
        row_1 = np.array([X[i], Y[i], 1, 0, 0, 0, -X[i]*u[i], -Y[i]*u[i], -u[i]])
        row_2 = np.array([0, 0, 0, X[i], Y[i], 1, -X[i]*v[i], -Y[i]*v[i], -v[i]])
        
        A.append(row_1)
        A.append(row_2)
        
    A = np.array(A)
    
    # V = eigvec(A.T @ A), being V.T obtained through Singular Value Decomposition (SVD)
    _, _, vT = np.linalg.svd(A)

  # vT is a 9×9 matrix
  # the solution x is the eigenvector corresponding to the smallest eigenvalue, 
  # that is, the eigenvector corresponding to the minimum singular value, 
  # leading to a row vector of 9 columns. Thus, to obtain the calibrated 
  # homography H, the final solution is to reshape the obtained vector into a 
  # 3x3 matrix 
    H = np.reshape(vT[-1,:], (3,3))
    
    # normalized homography, dividing by the element at (3,3)
    H = H/H[2,2]
    
    return H 

In [5]:
def get_errors(all_matches, H):
    """Compute error or distance between original points and transformed by H. 
   Return an array of errors for all points"""
    
    num_matches = len(all_matches)
    
    X = all_matches[:,0].reshape(-1, 1)
    Y = all_matches[:,1].reshape(-1, 1)
    u = all_matches[:,2].reshape(-1, 1)
    v = all_matches[:,3].reshape(-1, 1)
    
    # all matching points in source image
    all_p1 = np.concatenate((X, Y, np.ones((len(all_matches),1))), axis = 1)
    
    # all matching points in template image
    all_p2 = np.concatenate((u, v), axis = 1)
    
    # Transform every point in p1 to estimate p2
    estimate_p2homogeneous = H @ all_p1.T
    
    estimate_p2euclidean = (estimate_p2homogeneous/(estimate_p2homogeneous[-1]))[0:2]
    
    # Compute error of each matching pair
    errors = np.linalg.norm(all_p2 - estimate_p2euclidean.T, axis = 1) 
    
    return errors

In [6]:
def GetHomographyRANSAC(match_coords):
    
    """Function that computes linear (2D) Homography Calibration, implementing RANSAC
  for eliminating outliers and align correspondent matches. The main output concerns 
  a single transformation H that gets the most inliers in the course of all the 
  iterations. 
  
    Args:
        match_coords(numpy.ndarray): In dims (#matched pixels, 4).

    Returns:
        H(numpy.ndarray): Homography matrix, dims (3, 3).
    """
    
    global MAX_ITER, threshold_error
    
    N = 4 # four matches to initialize the homography in each iteration
    
    max_inliers = 0 
    
    # RANSAC procedure
    for itr in range(MAX_ITER): 
        # Randomly select 4 matched pairs
        idx_rand_inliers = random.sample(range(match_coords.shape[0]), N)
        
        selected_matches = match_coords[idx_rand_inliers, :]
        
        # compute the homography H by DLT from the N = 4 matched pairs 
        H = FitHomography(selected_matches)
        
        # Find inliners 
        errors = get_errors(match_coords, H)
        
        idx_inliers = np.where(errors < threshold_error)[0]
        
        num_inliers = len(idx_inliers) 
        
        # Analise current solution, and if it contains the maximum number of inliers
        # amongst all homographies until now fitted, save the current inliers for 
        # further refinement of the homography in the last step 
        
        if num_inliers > max_inliers:
            max_inliers = num_inliers
            best_inliers = match_coords[idx_inliers]
    
    # compute the homography H by DLT from best_inliers  
    H = FitHomography(best_inliers, max_inliers)
    
    return H

In [9]:
def compute_H_wrt_reference(H_all, ref_image):
    """
  Function that computes new homographies H_map that map every other image *directly* to
  the reference image by composing H matrices in H_all. 
  The homography in H_map that is associated with the reference image
  should be the identity matrix, created using eye(3). The homographies in
  H_map for the other images (both before and after the reference image)
  are computed by using already defined matrices in H_map and H_all. 

  Args: 
      H_all(cell array) 

      ref_image(int): index of the reference image (the first image has index 1)


  Returns:
      H_map(cell array): 3x3 homographies matrices that map each image into the reference image's
  coordinate system.

  """
    num_imgs = len(H_all)+1
    
    H_map = {}
    
    key = "H{}{}".format(ref_image-1, ref_image-1) 
    H_map[key] = np.eye(3)
    
    for i in range(0, ref_image-1): 
        key = "H{}{}".format(i, ref_image-1)  
        H_aux = np.eye(3)
        
        j = i
        
        while j < ref_image - 1:
            key_t = "H{}{}".format(j, j+1)
            H_aux = H_all[key_t] @ H_aux
            j += 1 
        
        H_map[key] = H_aux
    
    for i in range(ref_image, num_imgs):
        key = "H{}{}".format(i, ref_image-1)  # H10
        H_aux = np.eye(3)
        
        j = i -1 
        
        while j>= ref_image-1:
            key_t = "H{}{}".format(j, j+1)
            H_inv = np.linalg.inv(H_all[key_t])
            H_aux = H_inv/H_inv[2,2] @ H_aux
            j -= 1
        
        H_map[key] = H_aux
    
    return H_map 

In [10]:
def Check_Homography(H):
    """
  Check if homography is reasonable, according to certain criteria:
  - If the determinant of the homography det(H) is very close to 0, H is 
   close to singular;
  
  - If condition number of H (ratio of the first-to-last singular value) is
    infinite, the matrix H is singular, and if it is too large, H is 
    ill-conditioned. In non-mathematical terms, an ill-conditioned problem 
    is one where, for a small change in the inputs, there is a large 
    change in the output, that is, H is very sensitive to changes or errors 
    in the input. This means that the correct solution/answer to the 
    equation becomes hard to find;
        
  - If det(H) >>, the invert matrix would have a determinant too close 
    to 0, that is, H^-1 would be close to singular;
        
  - If det(H) < 0, the homography is not conserving the orientation, 
    being orientation-reversing. This is not suitable, except if we are 
    watching the object in a mirror. Nevertheless, sift/surf descriptors 
    are not done to be mirror invariant, so if it was the case we would 
    probably not have good maches. 
        
  An exactly singular matrix means that it is not invertible. If the above 
  criteria is verified, more pratically the matrix H or H^-1 is non-invertible. 
  In the context of homographies, it means that points in one 2D image are mapped
  to a less-than-2D subspace in the other image (a line, a point), i.e. 
  the estimated homography would be warping the image into nothing. A 
  nearly singular matrix is indicative of a rather extreme warp.  
  """
    
    #Conditions to accertain that the resultant homography H is free of 
    #singularities. If one of the condition is satisfied, the Homography H from 
    #image space to reference image space is not reasonable, according 
    #to the defined criteria 
    if  np.linalg.det(H) < 1 and np.linalg.cond(H[0:2, 0:2]) > 2:
        
        # In the condition number, only the top-left 2x2 matrix is considered, 
        # thus omitting the z-dependence of the transformation, which should be 
        # irrelevant because we know that z will always be fixed to 1 on the input
        
        H = np.zeros((3,3))
    
    return H

In [11]:
def pivproject2022_task2(ref_image, path_to_input_folder, path_to_output_folder):
    
    """
  Compute the homographies between images in a directory and a reference image

    ref_image: integer with index number of the frame that will be the reference 
    image. First image is index=1. Ideally, the one in the middle of the sequence 
    of input images so that there is less distortion resulting mosaic.
    
    path_to_input_folder: string with the path to the input folder, where input images 
                           and keypoints are stored. Images are named rgb_number.jpg 
                           (or rgb_number.png) and corresponding keypoints are named 
                           rgbsift_number.mat

    
    path_to_output_folder: string with the path where homographies with respect to the reference
    image are stored

  """
    
    # Check if path_to_input_folder was passed. If not, "No_path" is assigned
    if not('path_to_input_folder' in locals()):
        path_to_input_folder = "No_path";
    
    # Check if output directory exists. If not, output directory is created 
    if not(os.path.isdir(path_to_output_folder)):
        os.mkdir(path_to_output_folder)
    
    # Get input rgb images
    rgb_paths = []
    sift_paths = []
    
    for im_path in glob.glob(path_to_input_folder+'/*.jpg'):
        rgb_paths.append(im_path)
    
    if len(rgb_paths) == 0:
        print('ERROR: In the specified path there aren\'t image input files')
        return 
    
    else: 
        #Ordering the rgb_paths array, in such a way that consecutive frames follow 
        #each other  
        image_paths = sorted(rgb_paths)  
    
    print('Searching for sift .mat files')
        
    for im_path in glob.glob(path_to_input_folder+'/*.mat'):
        sift_paths.append(im_path)
    
    if len(sift_paths) != 0:
        sift_paths_ordered = sorted(sift_paths)
    
    else:
        print('ERROR: In the specified path there aren\'t sift input files.')
        return 
    
    # Get Reference image
    try: 
        reference_image = img.imread(image_paths[ref_image - 1])
        
    except:
        print('ERROR: The index for the reference image is out of bounds. Please select an index between 1 and %d' % (len(rgb_paths)))
        return
    
    H_all = {}
    # compute homography matrices between adjacent input images. Homography matrices 
    # between adjacent input images are then stored in a cell array H_all.
    
    for i in range(len(image_paths)-1):
        image_1_path = image_paths[i]
        image_2_path = image_paths[i+1]
        
        print("Processing {} & {}".format(image_1_path, image_2_path))
        
        image_1 = img.imread(image_1_path)
        image_2 = img.imread(image_2_path)
        
        sift_path1 = sift_paths_ordered[i]
        sift_path2 = sift_paths_ordered[i+1]
        
        key = 'H{}{}'.format(i, i+1)
        
        try:
            # coordinates of the matches between image and template 
            m_coords_img, m_coords_temp = siftMatch(image_1, sift_path2, sift_path1, N = 4)
            match_coords = np.append(m_coords_img, m_coords_temp, axis = 1)
        
        except:
            print('ERROR: check format of directory, as img.imread only accepts ASCII characters for image paths')
            return 
        
        try:
            
            H_all[key] = GetHomographyRANSAC(match_coords)
            
        except: 
            print('ERROR: RANSAC failed to compute homography. Check if there are enough matching keypoints.')
        
    # Compute new homographies H_map that map every other image *directly* to
    # the reference image 
    H_map = compute_H_wrt_reference(H_all, ref_image)
    
    for i in range(len(H_map)):
        key = "H{}{}".format(i, ref_image-1)
        H = H_map[key]
        
        print("image "+str(i))
        print("np.linalg.det(H):", np.linalg.det(H)) 
        print("np.linalg.cond(H[0:2, 0:2]):", np.linalg.cond(H[0:2, 0:2]))
            
        H = Check_Homography(H)

        if np.array_equal(H, np.zeros((3,3))):
            print("H{}{} is not reasonable".format(i, ref_image-1))
        else:
            print("H{}{} is reasonable".format(i, ref_image-1))

        # saving homographies with respect to the reference image 
        file_name = os.path.split(image_paths[i])[1]
        H_output_path = path_to_output_folder + '/' + 'H_' + file_name[4:8] + '.mat'
        scipy.io.savemat(H_output_path, {'H':H})
    
    print('All projections calculated.')


In [12]:
pivproject2022_task2(1, 'DATASETS/quarto', 'Output_folder_quarto')

Searching for jpg images
Searching for sift .mat files
Processing DATASETS/quarto/rgb_0001.jpg & DATASETS/quarto/rgb_0002.jpg
good matches/matches - 63/431
mutual and good matches/matches - 32/431
Processing DATASETS/quarto/rgb_0002.jpg & DATASETS/quarto/rgb_0003.jpg
good matches/matches - 35/573
mutual and good matches/matches - 20/573
image 0
np.linalg.det(H): 1.0
np.linalg.cond(H[0:2, 0:2]): 1.0
H00 is reasonable
image 1
np.linalg.det(H): 3.5710435736999515
np.linalg.cond(H[0:2, 0:2]): 1.6715923227010254
H10 is reasonable
image 2
np.linalg.det(H): 3.214570741778466
np.linalg.cond(H[0:2, 0:2]): 2.061826286240154
H20 is reasonable
All projections calculated.
